# User Spectrum Interpolator Demo

This notebook demonstrates how to use the `UserSpectrumInterpolator` class to load and interpolate your own grid of stellar spectra.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from spice.models import IcosphereModel
from spice.plots.plot_mesh import plot_3D
from spice.models.mesh_transform import add_rotation, evaluate_rotation
from spice.spectrum import simulate_observed_flux
from transformer_payne import TransformerPayne

import h5py
import pickle

# Import SPICE modules
from spice.spectrum.user_spectrum_interpolator import UserSpectrumInterpolator

## 1. Creating a Sample Spectrum Grid

First, let's create a sample grid of synthetic spectra to demonstrate the functionality. In a real-world scenario, you would use your own pre-computed spectra.

In [2]:
# Define a simple function to generate synthetic spectra based on parameters
def generate_synthetic_spectrum(wavelengths, teff, logg, metallicity, mu=1.0):
    """Generate a synthetic spectrum using a simplified model.
    
    Parameters:
    -----------
    wavelengths : array-like
        Wavelength array in Angstroms (Å)
    teff : float
        Effective temperature in Kelvin (K)
    logg : float
        Surface gravity in log(cm/s²)
    metallicity : float
        Metallicity [M/H] in dex
    mu : float, optional
        Limb darkening parameter (cosine of the angle between the line of sight and the normal)
    
    Returns:
    --------
    intensity : array-like
        Spectral intensity in arbitrary units (not physically calibrated)
    """
    # Physical constants
    h = 6.62607015e-27  # Planck's constant [erg*s]
    c = 2.99792458e10   # Speed of light [cm/s]
    k = 1.380649e-16    # Boltzmann constant [erg/K]
    
    # Convert wavelengths from Angstroms to meters
    wave_cm = wavelengths * 1e-8
    
    # Simplified Planck function (blackbody radiation)
    # Note: This produces intensity in arbitrary units, not physically calibrated
    intensity = ((2 * h * c ** 2 / wave_cm ** 5 * 1 / (np.exp(h * c / (wave_cm * k * teff)) - 1)))*1e-8
    
    # # Scale based on gravity (simplified)
    # # Higher gravity leads to slightly lower intensity in this simplified model
    # gravity_factor = np.exp(-(logg - 4.5)/5)
    # intensity *= gravity_factor
    
    # # Add metallicity effects (simplified)
    # # Higher metallicity increases line strength and affects continuum shape
    # metal_factor = 1.0 - 0.1 * metallicity * np.sin(2*np.pi*wavelengths/500)
    # intensity *= metal_factor
    
    # Add some absorption lines
    for line_center in [4000, 4500, 5000, 5500, 6000, 6500]:  # Line centers in Angstroms
        line_width = 5 * (1 + 0.2 * metallicity)  # Line width in Angstroms, wider with higher metallicity
        line_depth = 0.3 * (1 + 0.5 * metallicity) * np.exp(-(logg - 4.5)/3)  # Deeper with higher metallicity
        line_profile = line_depth * np.exp(-((wavelengths - line_center)/line_width)**2)
        intensity *= (1 - line_profile)
    
    return intensity

# Define the parameter grid
teff_values = [4000, 5000, 6000, 7000, 8000]  # Effective temperature in K
logg_values = [3.5, 4.0, 4.5, 5.0]            # Surface gravity (log g) in log(cm/s²)
metallicity_values = [-0.5, 0.0, 0.5]         # Metallicity [M/H] in dex
mu_values = [0.1, 0.3, 0.5, 0.7, 1.0]         # Limb darkening parameter (dimensionless)

# Create wavelength array (3500-7000 Å, 500 points)
wavelengths = np.linspace(3500, 7000, 500)

# Generate the grid
parameters = []
intensities = []

for teff in teff_values:
    for logg in logg_values:
        for metallicity in metallicity_values:
            for mu in mu_values:
                parameters.append([teff, logg, metallicity, mu])
                spectrum = generate_synthetic_spectrum(wavelengths, teff, logg, metallicity, mu)
                intensities.append(spectrum)

parameters = np.array(parameters)
intensities = np.array(intensities)
print(f"Generated {len(parameters)} spectra with shape {intensities.shape}")

Generated 300 spectra with shape (300, 500)


Let's visualize a few of the synthetic spectra to ensure they look reasonable:

In [3]:
# Plot a few spectra at different temperatures with fixed other parameters
plt.figure(figsize=(12, 5))

for teff in [4000, 6000, 8000]:
    # Find the index for a spectrum with this temperature and default other parameters
    idx = np.where((parameters[:, 0] == teff) & 
                   (parameters[:, 1] == 4.5) & 
                   (parameters[:, 2] == 0.0) &
                   (parameters[:, 3] == 1.0))[0][0]
    plt.plot(wavelengths, intensities[idx], label=f'Teff = {teff}K')

plt.xlabel('Wavelength (Å)')
plt.ylabel('Intensity (erg/s/cm²/Å/sr)')
plt.title('Sample Synthetic Spectra at Different Effective Temperatures')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_78974/169114651.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Now let's create a simple continuum spectrum to demonstrate the full capability of the interpolator:

In [4]:
# Generate continuum spectra (without absorption lines) using the same function
def generate_synthetic_continuum(wavelengths, teff, logg, metallicity, mu=1.0):
    """Generate a synthetic continuum spectrum (without lines)."""
    # Base continuum using blackbody approximation (simplified)
    h = 6.62607015e-27  # Planck's constant [erg*s]
    c = 2.99792458e10   # Speed of light [cm/s]
    k = 1.380649e-16    # Boltzmann constant [erg/K]
    
    # Convert wavelengths from Angstroms to meters
    wave_cm = wavelengths * 1e-8
    
    # Simplified Planck function (blackbody radiation)
    # Note: This produces intensity in arbitrary units, not physically calibrated
    intensity = ((2 * h * c ** 2 / wave_cm ** 5 * 1 / (np.exp(h * c / (wave_cm * k * teff)) - 1)))#*1e8
    
    # # Scale based on gravity (simplified)
    # gravity_factor = np.exp(-(logg - 4.5)/5)
    # intensity *= gravity_factor
    
    return intensity

# Generate continuum spectra
continuum_intensities = []
for teff, logg, metallicity, mu in parameters:
    spectrum = generate_synthetic_continuum(wavelengths, teff, logg, metallicity, mu)
    continuum_intensities.append(spectrum)

continuum_intensities = np.array(continuum_intensities)

from spice.spectrum.user_spectrum_interpolator import UserSpectrumInterpolator, linear_limb_darkening, quadratic_limb_darkening, nonlinear_limb_darkening

In [5]:
# Save grid as HDF5 file
h5_path = 'sample_spectrum_grid.h5'
with h5py.File(h5_path, 'w') as f:
    # Save parameters and spectra
    f.create_dataset('parameters', data=parameters)
    f.create_dataset('sp_intensity', data=intensities)
    f.create_dataset('log10_sp_wave', data=np.log10(wavelengths))
    
    # Save continuum spectra
    f.create_dataset('sp_no_lines_intensity', data=continuum_intensities)
    f.create_dataset('log10_sp_no_lines_wave', data=np.log10(wavelengths))
    
    # Add parameter names as attributes
    f.attrs['parameter_names'] = ['teff', 'logg', 'm/h', 'mu']

print(f"Saved grid to {h5_path}")

# Save grid as pickle file
pickle_path = 'sample_spectrum_grid.pkl'
grid_data = {
    'parameters': parameters,
    'intensities': intensities,
    'log10_wavelengths': np.log10(wavelengths),
    'continuum_intensities': continuum_intensities,
    'continuum_wavelengths': np.log10(wavelengths),
    'parameter_names': ['teff', 'logg', 'm/h', 'mu']
}

with open(pickle_path, 'wb') as f:
    pickle.dump(grid_data, f)

print(f"Saved grid to {pickle_path}")

Saved grid to sample_spectrum_grid.h5
Saved grid to sample_spectrum_grid.pkl


## 3. Using the Static Constructor Method

The `UserSpectrumInterpolator` class provides a static method to create a compatible grid file directly from raw data. Let's use that method as an alternative way to create our grid.

In [6]:
# Use the static method to create a grid file
static_h5_path = UserSpectrumInterpolator.create_grid_from_spectra(
    parameters=parameters.tolist(),
    wavelengths=wavelengths.tolist(),
    intensities=intensities.tolist(),
    continuum_wavelengths=wavelengths.tolist(),
    continuum_intensities=continuum_intensities.tolist(),
    parameter_names=['teff', 'logg', 'm/h', 'mu'],
    output_path='sample_spectrum_grid_static.h5'
)

print(f"Created grid file at {static_h5_path}")

Created grid file at sample_spectrum_grid_static.h5


## 4. Loading and Using the Interpolator

Now let's load our grid and demonstrate how to use the interpolator.

In [7]:
# Initialize the interpolator with our grid
interpolator = UserSpectrumInterpolator(
    grid_path=h5_path,
    parameter_names=['teff', 'logg', 'm/h'],  # Note: mu is handled separately
    has_continuum=True,
    has_mu=True
)

# Let's check the parameter space of our grid
print("Parameter ranges in the grid:")
print(f"Temperature (Teff): {np.min(interpolator.parameters[:, 0])} - {np.max(interpolator.parameters[:, 0])} K")
print(f"Surface gravity (log g): {np.min(interpolator.parameters[:, 1])} - {np.max(interpolator.parameters[:, 1])}")
print(f"Metallicity [M/H]: {np.min(interpolator.parameters[:, 2])} - {np.max(interpolator.parameters[:, 2])}")
print(f"Mu values: {np.unique(interpolator.parameters[:, 3])}")

Parameter ranges in the grid:
Temperature (Teff): 4000.0 - 8000.0 K
Surface gravity (log g): 3.5 - 5.0
Metallicity [M/H]: -0.5 - 0.5
Mu values: [0.1 0.3 0.5 0.7 1. ]


In [8]:
# Now let's use the interpolator to get spectra at arbitrary parameter values:

# Define some wavelengths to interpolate at (log10 scale)
log10_wavelengths = np.log10(np.linspace(3600, 6900, 100))

# Interpolate a spectrum at arbitrary parameter values
parameters1 = {'teff': 5500, 'logg': 4.2, 'm/h': 0.2}
mu1 = 0.8

# Get the interpolated spectrum
result1 = interpolator.intensity(log10_wavelengths, mu1, interpolator.to_parameters(parameters1))
spectrum1 = result1[:, 0]  # Line spectrum
continuum1 = result1[:, 1]  # Continuum spectrum

# Interpolate another spectrum with different parameters
parameters2 = {'teff': 7200, 'logg': 3.8, 'm/h': -0.2}
mu2 = 0.6
result2 = interpolator.intensity(log10_wavelengths, mu2, interpolator.to_parameters(parameters2))
spectrum2 = result2[:, 0]  # Line spectrum
continuum2 = result2[:, 1]  # Continuum spectrum

# Plot the interpolated spectra
plt.figure(figsize=(12, 6))

# Convert log10 wavelengths back to linear
wavelengths_to_plot = 10**log10_wavelengths

# Plot spectrum 1
plt.plot(wavelengths_to_plot, spectrum1, label=f'Spectrum 1 (Teff={parameters1["teff"]}, logg={parameters1["logg"]}, [M/H]={parameters1["m/h"]}, μ={mu1})')
plt.plot(wavelengths_to_plot, continuum1, '--', label='Continuum 1')

# Plot spectrum 2
plt.plot(wavelengths_to_plot, spectrum2, label=f'Spectrum 2 (Teff={parameters2["teff"]}, logg={parameters2["logg"]}, [M/H]={parameters2["m/h"]}, μ={mu2})')
plt.plot(wavelengths_to_plot, continuum2, '--', label='Continuum 2')

plt.xlabel('Wavelength (Å)')
plt.ylabel('Intensity (erg/s/cm²/Å/sr)')
plt.title('Interpolated Spectra')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_78974/861862095.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [9]:
# Define some wavelengths to interpolate at (log10 scale)
log10_wavelengths = np.log10(np.linspace(3600, 6900, 100))

# Interpolate a spectrum at arbitrary parameter values
parameters1 = {'teff': 5500, 'logg': 4.2, 'm/h': 0.2}
mu1 = 0.8

# Get the interpolated spectrum
result1 = interpolator.intensity(log10_wavelengths, mu1, interpolator.to_parameters(parameters1))
spectrum1 = result1[:, 0]  # Line spectrum
continuum1 = result1[:, 1]  # Continuum spectrum

# Interpolate another spectrum with different parameters
parameters2 = {'teff': 7200, 'logg': 3.8, 'm/h': -0.2}
mu2 = 0.6
result2 = interpolator.intensity(log10_wavelengths, mu2, interpolator.to_parameters(parameters2))
spectrum2 = result2[:, 0]  # Line spectrum
continuum2 = result2[:, 1]  # Continuum spectrum

# Plot the interpolated spectra
plt.figure(figsize=(12, 6))

# Convert log10 wavelengths back to linear
wavelengths_to_plot = 10**log10_wavelengths

# Plot spectrum 1
plt.plot(wavelengths_to_plot, spectrum1, label=f'Spectrum 1 (Teff={parameters1["teff"]}, logg={parameters1["logg"]}, [M/H]={parameters1["m/h"]}, μ={mu1})')
plt.plot(wavelengths_to_plot, continuum1, '--', label='Continuum 1')

# Plot spectrum 2
plt.plot(wavelengths_to_plot, spectrum2, label=f'Spectrum 2 (Teff={parameters2["teff"]}, logg={parameters2["logg"]}, [M/H]={parameters2["m/h"]}, μ={mu2})')
plt.plot(wavelengths_to_plot, continuum2, '--', label='Continuum 2')

plt.xlabel('Wavelength (Å)')
plt.ylabel('Intensity (erg/s/cm²/Å/sr)')
plt.title('Interpolated Spectra')
plt.legend()
plt.grid(alpha=0.3)

## 5. Integration with SPICE Mesh Models

Now let's demonstrate how to use our interpolator with SPICE mesh models to generate spectra for a rotating star.

In [10]:
mesh = IcosphereModel.construct(1000, 1., 1., interpolator.to_parameters(), interpolator.parameter_names)

mesh_rotated = evaluate_rotation(add_rotation(mesh, 250.), 1.)

# Plot the mesh to visualize the temperature spot
plt.figure(figsize=(8, 8))
plot_3D(mesh_rotated, 'los_velocities')
plt.show()

/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_78974/1498949308.py:8: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [11]:
spectrum = simulate_observed_flux(interpolator.intensity, mesh, log10_wavelengths)
spectrum_rotated = simulate_observed_flux(interpolator.intensity, mesh_rotated, log10_wavelengths)

plt.plot(10**log10_wavelengths, spectrum[:, 0])
plt.plot(10**log10_wavelengths, spectrum_rotated[:, 0])
plt.show()

/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_78974/297216148.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 6. Examining Line Profile Variations

Let's examine how a specific spectral line changes with rotation due to the temperature spot.

In [12]:
# Demo of using limb darkening with a grid that doesn't have mu values

# Create a version of our grid that doesn't include mu
no_mu_h5_path = 'sample_spectrum_grid_no_mu.h5'

# Get a subset of our data at mu=1.0 (disk center)
disk_center_mask = parameters[:, 3] == 1.0
disk_center_parameters = parameters[disk_center_mask, :3]  # Remove mu column
disk_center_intensities = intensities[disk_center_mask]
disk_center_continuum_intensities = continuum_intensities[disk_center_mask]

# Save to HDF5
with h5py.File(no_mu_h5_path, 'w') as f:
    f.create_dataset('parameters', data=disk_center_parameters)
    f.create_dataset('sp_intensity', data=disk_center_intensities)
    f.create_dataset('log10_sp_wave', data=np.log10(wavelengths))
    f.create_dataset('sp_no_lines_intensity', data=disk_center_continuum_intensities)
    f.create_dataset('log10_sp_no_lines_wave', data=np.log10(wavelengths))
    f.attrs['parameter_names'] = ['teff', 'logg', 'm/h']

print(f"Created no-mu grid at {no_mu_h5_path}")

Created no-mu grid at sample_spectrum_grid_no_mu.h5


In [13]:
# Initialize the no-mu interpolator with linear limb darkening
no_mu_interpolator = UserSpectrumInterpolator(
    grid_path=no_mu_h5_path,
    parameter_names=['teff', 'logg', 'm/h'],
    has_continuum=True,
    has_mu=False,
    limb_darkening_law="linear"
)

# Test the interpolator with different limb darkening coefficients
teff = 6000
logg = 4.5
metallicity = 0.0

# Test different mu values with a single linear coefficient
mus = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
linear_coeff = 0.6  # A typical value for solar-type stars

plt.figure(figsize=(12, 6))

for mu in mus:
    # Interpolate with limb darkening
    params = {'teff': teff, 'logg': logg, 'm/h': metallicity, 'limb_darkening': linear_coeff}
    result = no_mu_interpolator.intensity(log10_wavelengths, mu, params)
    spectrum = result[:, 0]  # Line spectrum
    
    plt.plot(10**log10_wavelengths, spectrum, label=f'μ={mu}')

plt.xlabel('Wavelength (Å)')
plt.ylabel('Intensity (erg/s/cm²/Å/sr)')
plt.title(f'Spectra with Linear Limb Darkening (coefficient={linear_coeff})')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

No mu values provided, using limb darkening law:  linear


/Users/mjablons/Documents/stellar-mesh-integration/src/spice/spectrum/user_spectrum_interpolator.py:375: UserWarning: No limb darkening coefficients provided. Using default values: 0.6
  warnings.warn(f"No limb darkening coefficients provided. Using default values: {limb_darkening_coeffs}")
/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_78974/2987922627.py:34: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [14]:
# Compare different limb darkening laws
mu_test = 0.5  # Test at μ=0.5

# Reinitialize with different limb darkening laws
interp_linear = UserSpectrumInterpolator(
    grid_path=no_mu_h5_path, has_mu=False, limb_darkening_law="linear")

interp_quadratic = UserSpectrumInterpolator(
    grid_path=no_mu_h5_path, has_mu=False, limb_darkening_law="quadratic")

interp_nonlinear = UserSpectrumInterpolator(
    grid_path=no_mu_h5_path, has_mu=False, limb_darkening_law="nonlinear")

# Define parameters with different limb darkening coefficients
params_linear = {'teff': teff, 'logg': logg, 'm/h': metallicity, 'limb_darkening': 0.6}
params_quadratic = {'teff': teff, 'logg': logg, 'm/h': metallicity, 'limb_darkening': (0.4, 0.2)}
params_nonlinear = {'teff': teff, 'logg': logg, 'm/h': metallicity, 'limb_darkening': (0.5, 0.2, 0.3, 0.1)}

# Get spectra
result_linear = interp_linear.intensity(log10_wavelengths, mu_test, params_linear)
result_quadratic = interp_quadratic.intensity(log10_wavelengths, mu_test, params_quadratic)
result_nonlinear = interp_nonlinear.intensity(log10_wavelengths, mu_test, params_nonlinear)

# Also get the mu=1.0 (disk center) spectrum for comparison
result_disk_center = interp_linear.intensity(log10_wavelengths, 1.0, params_linear)

# Plot comparison
plt.figure(figsize=(12, 6))
wavelengths_plot = 10**log10_wavelengths

plt.plot(wavelengths_plot, result_disk_center[:, 0], 'k--', label='Disk Center (μ=1.0)')
plt.plot(wavelengths_plot, result_linear[:, 0], label=f'Linear LD (μ={mu_test})')
plt.plot(wavelengths_plot, result_quadratic[:, 0], label=f'Quadratic LD (μ={mu_test})')
plt.plot(wavelengths_plot, result_nonlinear[:, 0], label=f'Nonlinear LD (μ={mu_test})')

plt.xlabel('Wavelength (Å)')
plt.ylabel('Intensity (erg/s/cm²/Å/sr)')
plt.title('Comparison of Limb Darkening Laws')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

No mu values provided, using limb darkening law:  linear
No mu values provided, using limb darkening law:  quadratic
No mu values provided, using limb darkening law:  nonlinear


/Users/mjablons/Documents/stellar-mesh-integration/src/spice/spectrum/user_spectrum_interpolator.py:375: UserWarning: No limb darkening coefficients provided. Using default values: (0.4, 0.2)
  warnings.warn(f"No limb darkening coefficients provided. Using default values: {limb_darkening_coeffs}")
/Users/mjablons/Documents/stellar-mesh-integration/src/spice/spectrum/user_spectrum_interpolator.py:375: UserWarning: No limb darkening coefficients provided. Using default values: (0.5, 0.2, 0.3, 0.1)
  warnings.warn(f"No limb darkening coefficients provided. Using default values: {limb_darkening_coeffs}")
/var/folders/7r/n_x0ntj511v_0gt816mgrc1c0000gq/T/ipykernel_78974/603548105.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Limb Darkening Support

The `UserSpectrumInterpolator` class now supports two different methods for handling limb darkening:

1. **Grid-based interpolation**: If your grid includes spectra at different μ values
2. **Analytical limb darkening**: If your grid only has disk-center spectra, you can apply analytical limb darkening laws

Three limb darkening laws are supported:
- Linear: I(μ)/I(1) = 1 - u(1-μ)
- Quadratic: I(μ)/I(1) = 1 - a(1-μ) - b(1-μ)²
- Non-linear (4-parameter): I(μ)/I(1) = 1 - a(1-μ½) - b(1-μ) - c(1-μ¹·⁵) - d(1-μ²)

This feature is useful when you have high-quality disk-center spectra but don't have angle-dependent intensity data.

In [15]:
# Visualize limb darkening across the stellar disk
mus = np.linspace(0.01, 1.0, 100)

# Calculate intensity factors for different limb darkening laws
linear_factor = linear_limb_darkening(mus, 0.6)
quadratic_factor = quadratic_limb_darkening(mus, (0.4, 0.2))
nonlinear_factor = nonlinear_limb_darkening(mus, (0.5, 0.2, 0.3, 0.1))

plt.figure(figsize=(10, 6))
plt.plot(mus, linear_factor, label='Linear (u=0.6)')
plt.plot(mus, quadratic_factor, label='Quadratic (a=0.4, b=0.2)')
plt.plot(mus, nonlinear_factor, label='Non-linear (4-param)')
plt.xlabel('μ = cos(angle)')
plt.ylabel('I(μ)/I(1)')
plt.title('Limb Darkening Laws')
plt.grid(alpha=0.3)
plt.legend()
plt.show()

# Create a 2D visualization of limb darkening on a stellar disk
def create_stellar_disk(radius=1.0, limb_darkening_func=None, ld_coeffs=None):
    """Create a 2D stellar disk with limb darkening"""
    x = np.linspace(-radius, radius, 500)
    y = np.linspace(-radius, radius, 500)
    X, Y = np.meshgrid(x, y)
    
    # Calculate distance from center
    r = np.sqrt(X**2 + Y**2)
    
    # Create mask for points inside the disk
    mask = r <= radius
    
    # Initialize intensity array
    intensity = np.zeros_like(r)
    
    # For points inside the disk, calculate mu
    mu = np.sqrt(1 - (r[mask]/radius)**2)
    
    # Apply limb darkening
    if limb_darkening_func and ld_coeffs is not None:
        intensity[mask] = limb_darkening_func(mu, ld_coeffs)
    else:
        intensity[mask] = 1.0
    
    return intensity

# Create disks with different limb darkening laws
disk_linear = create_stellar_disk(limb_darkening_func=linear_limb_darkening, ld_coeffs=0.6)
disk_quadratic = create_stellar_disk(limb_darkening_func=quadratic_limb_darkening, ld_coeffs=(0.4, 0.2))
disk_nonlinear = create_stellar_disk(limb_darkening_func=nonlinear_limb_darkening, ld_coeffs=(0.5, 0.2, 0.3, 0.1))

# Plot the disks
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(disk_linear, cmap='viridis', origin='lower')
axes[0].set_title('Linear Limb Darkening')
axes[0].set_axis_off()

axes[1].imshow(disk_quadratic, cmap='viridis', origin='lower')
axes[1].set_title('Quadratic Limb Darkening')
axes[1].set_axis_off()

axes[2].imshow(disk_nonlinear, cmap='viridis', origin='lower')
axes[2].set_title('Non-linear Limb Darkening')
axes[2].set_axis_off()

plt.tight_layout()
plt.show()

NameError: name 'linear_limb_darkening' is not defined

In [1]:
# Choose a wavelength range around one of our absorption lines (e.g., around 5000Å)
line_center = 5000
line_width = 100
line_range = (line_center - line_width, line_center + line_width)

# Filter the wavelengths to focus on our chosen line
line_mask = (wavelengths >= line_range[0]) & (wavelengths <= line_range[1])
line_wavelengths = wavelengths[line_mask]

plt.figure(figsize=(10, 6))

for i, angle in enumerate(rotation_angles):
    # Extract the line profile and its continuum
    line_spectrum = spectra[i][line_mask]
    line_continuum = continua[i][line_mask]
    
    # Normalize by the continuum to better show the line profile variations
    normalized_spectrum = line_spectrum / line_continuum
    
    plt.plot(line_wavelengths, normalized_spectrum, label=f'Rotation {angle}°')

plt.xlabel('Wavelength (Å)')
plt.ylabel('Normalized Flux')
plt.title('Line Profile Variations with Rotation')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

NameError: name 'wavelengths' is not defined

## 7. Creating an Animation of Spectral Variations with Rotation

Finally, let's create an animation to visualize how the spectrum changes as the star rotates.

In [ ]:
from matplotlib.animation import FuncAnimation

# Generate more rotation angles for a smoother animation
n_frames = 36
animation_angles = np.linspace(0, 360, n_frames, endpoint=False)

# Pre-compute spectra for all frames
animation_spectra = []
animation_continua = []

for angle in animation_angles:
    angle_rad = np.radians(angle)
    spectrum, continuum = generate_integrated_spectrum(log10_wavelengths, mesh, teff, logg, metallicity, angle_rad)
    animation_spectra.append(spectrum)
    animation_continua.append(continuum)

# Create the animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
fig.subplots_adjust(wspace=0.3)

# Initialize plots
line1, = ax1.plot([], [], 'b-')
line2, = ax2.plot([], [], 'r-')

ax1.set_xlim(wavelengths.min(), wavelengths.max())
ax1.set_ylim(min([spec.min() for spec in animation_spectra]) * 0.9, 
            max([spec.max() for spec in animation_spectra]) * 1.1)
ax1.set_xlabel('Wavelength (Å)')
ax1.set_ylabel('Flux')
ax1.set_title('Full Spectrum')
ax1.grid(alpha=0.3)

# Set up the line profile plot
ax2.set_xlim(line_range[0], line_range[1])
# Calculate the range for normalized profiles
norm_min = min([min(animation_spectra[i][line_mask] / animation_continua[i][line_mask]) for i in range(n_frames)]) * 0.99
norm_max = min([max(animation_spectra[i][line_mask] / animation_continua[i][line_mask]) for i in range(n_frames)]) * 1.01
ax2.set_ylim(norm_min, norm_max)
ax2.set_xlabel('Wavelength (Å)')
ax2.set_ylabel('Normalized Flux')
ax2.set_title('Line Profile')
ax2.grid(alpha=0.3)

# Title for the overall figure
fig.suptitle('Spectrum Variations with Stellar Rotation', fontsize=16)

# Text annotation for rotation angle
angle_text = ax1.text(0.05, 0.95, '', transform=ax1.transAxes, va='top')

def update(frame):
    # Update full spectrum
    line1.set_data(wavelengths, animation_spectra[frame])
    
    # Update line profile
    normalized_profile = animation_spectra[frame][line_mask] / animation_continua[frame][line_mask]
    line2.set_data(line_wavelengths, normalized_profile)
    
    # Update rotation angle text
    angle_text.set_text(f'Rotation: {animation_angles[frame]:.1f}°')
    
    return line1, line2, angle_text

# Create animation
anim = FuncAnimation(fig, update, frames=n_frames, interval=100, blit=True)

# Display the animation
from IPython.display import HTML
HTML(anim.to_jshtml())

## 8. Conclusion

This notebook has demonstrated how to:

1. Create a sample grid of synthetic spectra
2. Save the grid in both HDF5 and pickle formats
3. Load the grid using the `UserSpectrumInterpolator` class
4. Interpolate spectra at arbitrary stellar parameters
5. Integrate the interpolator with SPICE mesh models to generate spectra for a rotating star with a spot
6. Visualize line profile variations due to the spot

The `UserSpectrumInterpolator` provides a flexible framework for using your own pre-computed grid of spectra with the SPICE ecosystem, allowing for detailed spectral synthesis of complex stellar surfaces.